## 基礎使用

官方文件:
https://python.langchain.com/docs/introduction/



In [1]:
# 導入 openai api key
import os
from dotenv import load_dotenv
load_dotenv()  # 加載 .env 文件中的環境變量
# 或是直接填
#os.environ["OPENAI_API_KEY"] = 'sk-'
#os.environ["SERPAPI_API_KEY"] = '1c'

# 導入 openai 的 api
from langchain_openai import ChatOpenAI

# 創建 openai 的 api 物件
llm = ChatOpenAI(model_name="gpt-4o-mini")

In [3]:
# temperature 介於 0~1，越大回答越發散，反之則越固定
# 以下是最基礎的使用方式
llm("請告訴我台灣是不是國家", temperature=0.6)

AIMessage(content='台灣的地位是一個複雜且敏感的問題。在事實上，台灣擁有自己的政府、軍隊、法律系統和經濟體系，並且有許多國家與台灣保持非正式的外交和經濟關係。然而，中國大陸視台灣為其領土的一部分，並且在國際上推動「一個中國」政策，這使得台灣的國際承認受到限制。\n\n因此，是否將台灣視為一個國家，取決於不同的政治立場和國際關係的背景。在台灣，許多人自認為是台灣人，並希望維護台灣的現狀或尋求更大的國際承認。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 170, 'prompt_tokens': 17, 'total_tokens': 187, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'stop', 'logprobs': None}, id='run-e4fa361f-d8d9-444b-a585-de05ff212efd-0', usage_metadata={'input_tokens': 17, 'output_tokens': 170, 'total_tokens': 187, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## 使用工具

範例以 Google Search 為例

需要安裝的套件
* pip install google-search-results
* pip install langchain-community


先到 https://serpapi.com/ 申請一組 api key



### LangChain initialize_agent 說明
initialize_agent 用於初始化一個 agent executor，將工具（tools）和語言模型（LLM）組合在一起，創建一個能執行任務的智能代理。

#### 必要參數
1. tools: Sequence[BaseTool]
    * 代理可以使用的工具列表

2. llm: BaseLanguageModel
    * 用作代理的語言模型

#### 可選參數
1. agent: Optional[AgentType]

    代理類型

    預設值：ZERO_SHOT_REACT_DESCRIPTION
    
    * 常見類型：
        * ZERO_SHOT_REACT_DESCRIPTION：零樣本反應式代理 (一個按照「思考→行動→觀察」簡單循環來解決問題的 AI，不需要預先學習就能直接工作。)
        * STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION：結構化對話零樣本反應式代理 (在零樣本反應式代理的基礎上，增加了處理複雜指令和多步驟任務的能力，並能提供更有條理的回答。)

2. callback_manager: Optional[BaseCallbackManager]
    
    回調管理器

3. verbose: Optional[Bool]

    在 LangChain 中，verbose 參數的作用是控制是否顯示 AI 的思考過程

In [18]:
# 1. 基本設置：讀取環境變數
from dotenv import load_dotenv
load_dotenv()  # 從.env檔案讀取重要的密鑰，這樣比較安全
# 或是直接填
#os.environ["OPENAI_API_KEY"] = 'sk-'
#os.environ["SERPAPI_API_KEY"] = '1c'

# 2. 設置搜尋工具
# 這就像給AI一個可以上網搜尋的能力
from langchain.agents import load_tools
from langchain.agents import AgentType
tools = load_tools(["serpapi"])  # 載入 Google 搜尋工具

#-------------------------------------------------------------#

# 3. 設置OpenAI模型
from langchain_openai import ChatOpenAI
# 創建一個AI助手，temperature=0表示回答會比較固定和準確
llm = ChatOpenAI(temperature=0)

#-------------------------------------------------------------#

# 4. 初始化代理
# 創建一個可以使用工具的AI代理
from langchain.agents import initialize_agent
# verbose=True 表示會顯示AI的思考過程
agent = initialize_agent(
    tools=tools, 
    llm=llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True
)

# 使用方式：
# agent.run("你的問題")
# 這樣就可以讓AI助手幫你完成任務了！

In [21]:
agent.run("使用繁體中文回答：請問 2025 年台灣的過年連假在國曆幾月幾號開始跟結束")



> Entering new AgentExecutor chain...
我應該使用搜索引擎來查找2025年台灣過年連假的日期
Action: Search
Action Input: "2025年台灣過年連假日期"
Observation: {'type': 'organic_result', 'title': '2025行事曆》2025過年放9天！114年連假/補班/請假攻略一次看', 'source': 'KKday', 'description': '2025行事曆：台灣請假攻略'}
Thought:我應該點進這個連結查看具體日期
Action: Search
Action Input: "2025行事曆 台灣過年連假日期"
Observation: ['2025 年共有六個連假，全年休假日115 天。本文將教你如何請假才能擁有3 個9 天連假，並推薦連假旅遊好去處！⭐春節年假：1/25（六）至2/2（日），共9 天。⭐228 連假：2/28（五） ...', '2025端午節連假3天 ... 因端午節適逢周六，周五5月30日將會補假1日。 端午節請假攻略： ... 若明年多請5月26日至5月29日4天假，可自5月24日周六開始放假，連續休假 ...', '2025行事曆出爐！補班1天、過年連假9天｜附錄114年請假攻略 ; 2025連假名稱. 連假天數. 2025連假日期 ; 農曆春節連假. 9天, 01/25六－02/02日 ; 228和平紀念日連假. 3天, 02/28五 ...', '2025行事曆｜2025連假請假攻略 ; 1 月元旦. 1/1(三). 共1天 ; 1 月春節連假. 1/25(六) ~ 2/2(日) ; 2 月228 連假. 2/28(五) ~ 3/2(日). 共3 天 ; 4 月清明連假.', '2025過年，含小年夜的彈性放假、從除夕到初三的國定假日，以及假期前後週末，今年春節假期，可以連續放到9天！ 2025寒假時間？ 2025年國小寒假預計落在1/13～2/2 ...', '2025(114年)行事曆搶先看…2025年過年放9天！3天以上連假6個、請假攻略必看：狂休13天 ; 1/24（五）. 自行請假 ; 1/25（六） ; 1/26（日） ; 1/27（一）. 小年夜彈性放假.', '2025過年休幾天？114年春

'2025年台灣的過年連假在國曆1月25日開始，2月2日結束'

## ReAct + 搜尋工具

其中會用到 LangSmith 上提供的預設 prompt

這邊的範例是由 hwchase17 提供

內容如下

https://smith.langchain.com/hub/hwchase17/react

```
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
```

可以不需要用`hub.pull`引入直接 = 上面那串字串，然後自行修改，也OK

###  create_react_agent 說明
1. llm: BaseLanguageModel
    * 用作代理的語言模型

2. tools: Sequence[BaseTool]
    * 這是代理可以使用的工具列表
    * 每個工具都是一個繼承自 BaseTool 的物件
    * 工具可以是搜尋引擎、計算器、API 呼叫等具體功能

3. prompt: BasePromptTemplate
    * 定義了代理使用的提示模板
    * 必須包含三個關鍵輸入變數：
        * tools: 工具的描述和參數
        * tool_names: 所有工具的名稱
        * agent_scratchpad: 代理之前的操作和工具輸出記錄

4. output_parser: Optional[AgentOutputParser]
    * 輸出解析器
    * 用於解析代理的輸出
    * 預設值：ReActSingleInputOutputParser

5. tools_renderer: ToolsRenderer
    * 控制如何將工具轉換為字串形式
    * 預設使用 render_text_description 函數
    * 這個字串描述會被傳入 LLM

6. stop_sequence: Union[bool, List[str]]
    * 控制停止標記的使用
    * 如果為 True：添加 "\nObservation" 作為停止標記
    * 如果為 False：不使用停止標記
    * 如果是字串列表：使用提供的列表作為停止標記
    * 主要目的是：
        * 防止 LLM 自己生成假的觀察結果
        * 如果不設置這個 stop sequence：
            * 可能的問題情況
            * Thought: 我需要搜索鈊象電子的資訊
            * Action: Search
            * Action Input: 鈊象電子 最新消息
            * Observation: 我發現鈊象電子最近...  # <-- LLM 可能會自己編造觀察結果



In [37]:
# 1. 基本設置：讀取環境變數
from dotenv import load_dotenv
load_dotenv()  # 從.env檔案讀取重要的密鑰（比如API金鑰），這樣比較安全

# 2. 設置AI模型
# LangChain Hub是一個存放預設設定的地方，就像是一個工具箱
from langchain import hub
# 取得ReAct的預設設定（ReAct是一種讓AI思考-行動-觀察的方法）
ReActPrompt = hub.pull("hwchase17/react")
print(ReActPrompt)

#-------------------------------------------------------------#

# 3. 設置OpenAI聊天模型
from langchain_openai import ChatOpenAI
# 創建一個AI助手，就像幫你請了一個智能小幫手
llm = ChatOpenAI(
    # 選擇使用哪個AI模型，這裡用的是比較強的GPT-4
    model_name="gpt-4-turbo-preview",
    # temperature是控制AI回答的創意程度，0.1表示回答會比較固定和準確
    temperature=0.1
)

#-------------------------------------------------------------#

# 4. 設置搜尋工具
# 這就像給AI一個可以上網搜尋的能力
from langchain_community.utilities import SerpAPIWrapper
from langchain.tools import Tool
# 創建一個搜尋器
search = SerpAPIWrapper()
# 把搜尋功能包裝成一個工具，就像給AI一個可以使用的應用程式
tools = [
    Tool(
        name="Search",  # 工具的名字
        func=search.run,  # 工具的功能
        description="用於搜尋即時資訊"  # 告訴AI這個工具是用來做什麼的
    ),
]

#-------------------------------------------------------------#

# 5. 創建AI代理
# ReAct代理是一種特殊的AI，它會先思考該怎麼做，再採取行動，最後觀察結果
from langchain.agents import create_react_agent
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.tools.render import render_text_description

# 把AI模型和工具組合在一起，創建一個更完整的助手
agent = create_react_agent(
    llm=llm,  # 語言模型
    tools=tools,  # 工具列表
    prompt=ReActPrompt,  # 提示模板
    
    # 可選：自定義輸出解析器
    output_parser=ReActSingleInputOutputParser(),

    # 可選：自定義工具渲染器，控制工具描述如何呈現給LLM render_text_description（默認）
    # 工具會被呈現為簡單的文字描述
    # """
    # Search: useful for searching the internet to find information about current events and data
    # Calculator: useful for performing mathematical calculations
    # """
    tools_renderer=render_text_description,
)
#-------------------------------------------------------------#

# 6. 設置執行環境
from langchain.agents import AgentExecutor
# 創建一個可以執行任務的環境，讓AI助手能夠實際工作
# verbose=True 表示會顯示AI的思考過程
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# 使用方式：
# agent_executor.invoke({"input": "你的問題"})
# 這樣就可以讓AI助手幫你完成任務了！

c:\Users\pochenkuo\AppData\Local\anaconda3\envs\py39\lib\site-packages\langsmith\client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'} template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'


In [38]:
# 執行兩次相同的查詢來測試回答的一致性
# 第一次執行
print("第一次運行的結果：")
agent_executor.invoke({
    "input": "使用繁體中文回答： 當前鈊象電子的公司狀況如何? 請務必用中文回覆。"
})

# 第二次執行相同的查詢
# 這樣做可以：
# 1. 驗證回答的穩定性
# 2. 檢查是否每次都會重新搜索
# 3. 比較兩次回答的差異
print("第二次運行的結果：")
agent_executor.invoke({
    "input": "使用繁體中文回答： 當前鈊象電子的公司狀況如何? 請務必用中文回覆。"
})

第一次運行的結果：


> Entering new AgentExecutor chain...
Thought: 我需要查找最新的關於鈊象電子的公司狀況信息。
Action: Search
Action Input: 鈊象電子 最新消息 2023['https://www.igs.com.tw/news/', 'https://www.igs.com.tw/', 'https://www.igs.com.tw/cn/news/45/', 'https://winvest.tw/Stock/Symbol/Comment/3293', 'http://mp.cnfol.com/55038/article/1710136087-141272947.html', 'https://hk.finance.yahoo.com/news/%E7%87%9F%E6%94%B6%E9%80%9F%E5%A0%B1-%E9%88%8A%E8%B1%A1-3293-10%E6%9C%88%E7%87%9F%E6%94%B616-80%E5%84%84%E5%85%83%E5%B9%B4%E5%A2%9E%E7%8E%87%E9%AB%98%E9%81%9436-083031030.html', 'https://uanalyze.com.tw/articles/368136980', 'https://www.igs.com.tw/investors/reports/64/', 'https://www.chinatimes.com/realtimenews/20240307005541-260410', 'https://www.cmoney.tw/forum/stock/3293']我需要點開一些最新的新聞或報導來獲取鈊象電子的最新公司狀況。
Action: Search
Action Input: 鈊象電子 最新財報 2023{'type': 'organic_result', 'title': '月營收 - 鈊象電子', 'source': '鈊象電子股份有限公司', 'description': '2023'}我發現了一個直接來自鈊象電子官方網站的連結，看起來像是提供了2023年的月營收資訊，但我需要更具體的資訊來回答這個問題。
Action: Search
Action 

{'input': '使用繁體中文回答： 當前鈊象電子的公司狀況如何? 請務必用中文回覆。',
 'output': '鈊象電子目前的狀況良好。根據2023年的資料，鈊象電子已經成立三十餘年，目前擁有超過1,000名員工。在2023年度，公司的合併營收達到了141億元，合併稅後淨利為64億元。公司表示將持續開發創意休閒產品並厚植遊戲產品研發能力，以實現企業的永續經營並與夥伴共享共榮。'}

In [36]:
# 我們稍微改一些參數
# 其中 output_parser 還可以使用:
# # 1. 標準 ReAct 解析器
# from langchain.agents.output_parsers import ReActSingleInputOutputParser
# output_parser = ReActSingleInputOutputParser()

# # 2. 結構化輸出解析器
# from langchain.agents.output_parsers import JSONAgentOutputParser
# output_parser = JSONAgentOutputParser()

# # 3. XML 格式解析器
# from langchain.agents.output_parsers import XMLAgentOutputParser
# output_parser = XMLAgentOutputParser()

## 以下以 json 解析器做為示範

# 5. 創建AI代理
# ReAct代理是一種特殊的AI，它會先思考該怎麼做，再採取行動，最後觀察結果
from langchain.agents import create_react_agent
from langchain.tools.render import  render_text_description_and_args
from langchain.agents.output_parsers import JSONAgentOutputParser

# 創建新的 prompt 模板
# https://smith.langchain.com/hub/hwchase17/react-multi-input-json <- 其中規範了 AI 必須以 json 格式作為輸出
json_prompt = hub.pull("hwchase17/react-multi-input-json")

# 把AI模型和工具組合在一起，創建一個更完整的助手
agent2 = create_react_agent(
    llm=llm,  # 語言模型
    tools=tools,  # 工具列表
    prompt=json_prompt,  # 提示模板
    
    # 可選：自定義輸出解析器
    output_parser=JSONAgentOutputParser(),
)
#-------------------------------------------------------------#

# 6. 設置執行環境
from langchain.agents import AgentExecutor
# 創建一個可以執行任務的環境，讓AI助手能夠實際工作
# verbose=True 表示會顯示AI的思考過程
# 設置執行環境
agent_executor2 = AgentExecutor(
    agent=agent2, 
    tools=tools, 
    verbose=True,
    handle_parsing_errors=True  # 保留錯誤處理
)

# 執行兩次相同的查詢來測試回答的一致性
# 第一次執行
print("第一次運行的結果：")
agent_executor2.invoke({
    "input": "使用繁體中文回答： 當前鈊象電子的公司狀況如何? 請務必用中文回覆。"
})

# 第二次執行相同的查詢
# 這樣做可以：
# 1. 驗證回答的穩定性
# 2. 檢查是否每次都會重新搜索
# 3. 比較兩次回答的差異
print("第二次運行的結果：")
agent_executor2.invoke({
    "input": "使用繁體中文回答： 當前鈊象電子的公司狀況如何? 請務必用中文回覆。"
})

c:\Users\pochenkuo\AppData\Local\anaconda3\envs\py39\lib\site-packages\langsmith\client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


第一次運行的結果：


> Entering new AgentExecutor chain...
{
  "action": "Search",
  "action_input": "鈊象電子 公司狀況 2023"
}鈊象至今成立三十餘年，已擁有1,000多名員工，在團隊的努力下，2023年度合併營收141億元，合併稅後淨利64億元，未來我們將持續 開發深具創意的休閒產品，厚植遊戲產品研發能力，使企業永續經營，攜手夥伴共享共榮。```
{
  "action": "Final Answer",
  "action_input": "鈊象電子目前的公司狀況相當穩健。成立三十餘年來，公司已發展至擁有1,000多名員工。在2023年度，鈊象電子的合併營收達到141億元，合併稅後淨利為64億元。公司未來計畫持續開發創新的休閒產品，並加強遊戲產品的研發能力，以期達到企業的永續經營，並與合作夥伴共享成功。"
}
```

> Finished chain.
第二次運行的結果：


> Entering new AgentExecutor chain...
{
  "action": "Search",
  "action_input": "鈊象電子 最新消息"
}['所有訊息 · 27 2023/11. 《鈊象電子》獻上經典與創新遊戲饗宴點燃2023漫畫博覽會熱潮 · 27 2023/11. 鈊象奧特曼VR系列奪原創遊戲優勝.', '距離美股開盤還有0729. 更新. 鈊象. 3293. PK. 加入自選股. 983. 2.00(0.20%). 收盤| 2025/01/06 13:30 更新. 成交量1,231; 本益比(同業平均)34.30 (50.76); 連漲連跌.', '最新消息 · 鈊象好生活 · 加入鈊象 · 職涯發展. Recruit. 人才招募. 人才是鈊象最重視的資產，擁有寬敞舒適的辦公環境，提供專業的設備 ...', '今日鈊象电子股票(3293)行情，实时最新价格，走势图表，及鈊象电子(3293)股票的专业技术分析，历史数据，最新消息和未来股价预测。', '迎接2025年七檔拚進千金俱樂部 · 鈊象犀利11月每股賺3.66元 · 鈊象營收寫最旺11月 · 鈊象營收／11月16.71億元創同期新高連二個月站上16億元大關 · 台股有望重現19千金榮景法人 

{'input': '使用繁體中文回答： 當前鈊象電子的公司狀況如何? 請務必用中文回覆。',
 'output': '鈊象電子目前的公司狀況看起來相當積極。最近的消息顯示，鈊象電子在2023年漫畫博覽會上獻上了經典與創新的遊戲饗宴，並且其奧特曼VR系列在原創遊戲中奪得優勝。此外，鈊象電子的股票表現也相當不錯，最近的股價為981元，單日上漲1.87%，顯示出市場對鈊象電子的正面看法。公司在10月的營收達到16.8億元，年增率達30.25%，且單月稅前每股盈餘為3.6元。特別值得一提的是，海外市場的營收授權比例已從第三季的56%進一步提升，顯示出鈊象電子在國際市場上的業務擴展正在穩步進行。'}